In [1]:
import yfinance as yf
import pandas as pd

# 티커와 그에 해당하는 이름을 정의합니다.
# 사용자는 이 딕셔너리에 새로운 티커와 이름을 추가할 수 있습니다.
tickers = {
    "CL=F": "WTI",
    "USDKRW=X": "USDKRW",
    "^KS11": "KOSPI200",
    "^VIX": "VIX"
}

data = pd.DataFrame()  # 빈 DataFrame을 초기화합니다.

# 각 티커에 대한 데이터를 다운로드하고 단일 DataFrame으로 병합합니다.
for ticker, name in tickers.items():
    series = yf.download(ticker, start="2006-12-30", end="2023-09-21")['Close']  # 각 티커에 대한 종가 데이터를 다운로드합니다.
    series.name = name  # 다운로드한 시리즈의 이름을 설정합니다.
    if data.empty:  # 첫 번째 티커의 경우, data DataFrame을 초기화합니다.
        data = pd.DataFrame(series).reset_index()
    else:  # 그 이후의 티커에 대해서는 data DataFrame과 병합합니다.
        data = pd.merge(data, pd.DataFrame(series).reset_index(), on='Date', how='outer')

# 'Date' 컬럼의 이름을 변경하고 인덱스로 설정합니다.
data.rename(columns={'Date': 'Date'}, inplace=True)
data.set_index('Date', inplace=True)

# 누락된 데이터를 처리합니다.
data.fillna(method='ffill', inplace=True)  # 누락된 데이터를 앞쪽으로 채웁니다.
data.fillna(method='bfill', inplace=True)  # 시작 부분의 누락된 데이터를 뒤쪽으로 채웁니다.

# KOSPI 200에 대한 포워드 스테이지를 계산합니다.
forward_days = 60
data['Forward_Return'] = data['KOSPI200'].shift(-forward_days) / data['KOSPI200'] - 1  # 포워드 리턴을 계산합니다.
data['forward_stage'] = pd.cut(data['Forward_Return'], bins=[-float('inf'), 0, 0.04, float('inf')], labels=['down', 'neutral', 'up'])  # 포워드 리턴을 기반으로 스테이지를 분류합니다.

# 'forward_stage'에서 NaN이 있는 행을 삭제합니다.
data.dropna(subset=['forward_stage'], inplace=True)

# 숫자형 컬럼을 2소수점 자리로 반올림합니다.
numerical_columns = [name for name in tickers.values() if name != 'KOSPI200']  # 'KOSPI200'을 제외한 모든 컬럼을 선택합니다.
data[numerical_columns] = data[numerical_columns].round(2)  # 선택한 컬럼을 반올림합니다.

# 인덱스를 재설정하고 'Date' 컬럼을 형식화합니다.
data.reset_index(inplace=True)
data['Date'] = data['Date'].dt.strftime('%y-%m-%d')

# 'Forward_Return' 컬럼을 삭제하고 컬럼 순서를 재배열합니다.
data = data[['Date', 'forward_stage'] + numerical_columns]

# 필요한 경우 CSV로 저장합니다.
data.to_csv('daily_data_value.csv', index=False)

# DataFrame의 처음 몇 행을 출력합니다.
print(data.head())


In [1]:
print("1")

1


In [1]:
# -*- coding: utf-8 -*- 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn.metrics as mt 
from sklearn.tree import export_graphviz 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV 
import joblib 
from sklearn.metrics import confusion_matrix 

# 1. Load the modified CSV data
model_data = pd.read_csv("feature.csv", encoding='cp949')

# Set 'Date' column as index
model_data.set_index('Date', inplace=True)

# 2. Generate complete data of features and label
X = model_data.drop(columns=['forward_stage'])  # Exclude the 'forward_stage' column for features
y = model_data['forward_stage']  # Use 'forward_stage' column as label

X_past = X[y.notna()] 
y_past = y[y.notna()]

# 3. Split the data into train and test
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_past, y_past):
    X_train, X_test = X_past.iloc[train_index,], X_past.iloc[test_index,] 
    y_train, y_test = y_past.iloc[train_index], y_past.iloc[test_index]

# 4. Model fine-tuning: find optimal hyperparameters
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42)  # n_jobs를 1로 설정

param_dist_rf = {
    'n_estimators': [50, 100, 500],
    'max_leaf_nodes': [20, 30, 40, 50],
    'max_features': [1, 2, 3],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42)
rnd_search.fit(X_train, y_train)
print(rnd_search.best_params_)

# 5. Train the model and evaluate it using K-fold cross-validation
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42, n_jobs=-1)  # n_jobs를 1로 설정
rnd_scores = cross_val_score(rnd_clf, X_train, y_train, scoring="accuracy", cv=10)
print("\n<10-fold cross-validation>")
print("accuracy score mean: ", rnd_scores.mean())

# 6. Train the final model
rnd_clf.fit(X_train, y_train)
print("\n<AI model: machine learning done >")
print("accuracy_score of train data(0.8 of sample): ", rnd_clf.score(X_train, y_train))

# 7. Evaluate the model on test data
print("accuracy_score of test data(0.2 of sample): ", rnd_clf.score(X_test, y_test))

# 8. Check the confusion matrix
y_test_pred = rnd_clf.predict(X_test) 
cm1 = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"]) 
print("\n<Confusion matrix>")
print("(of test)")
print("up", "neutral", "down")
print(cm1)
cm2 = confusion_matrix(y_past, rnd_clf.predict(X_past), labels=["up", "neutral", "down"]) 
print("(of all)")
print("up", "neutral", "down")
print(cm2)

# 9. Check feature importance
print("\n<Feature importance>")
for name, score in zip(X.columns, rnd_clf.feature_importances_):
    print(name, ": ", score)

# 10. Generate prediction data for backtesting
y_prediction = rnd_clf.predict(X)
y_pred = pd.Series(y_prediction, index=y.index)

# 11. Save the model
joblib.dump(rnd_clf, "forecast_model.pkl")
print("\n< AI model: save >")


{'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_leaf_nodes': 50, 'max_features': 3, 'max_depth': 10}

<10-fold cross-validation>
accuracy score mean:  0.8576162989336641

<AI model: machine learning done >
accuracy_score of train data(0.8 of sample):  0.9976019184652278
accuracy_score of test data(0.2 of sample):  0.8610778443113772

<Confusion matrix>
(of test)
up neutral down
[[266  24   3]
 [ 26 132  31]
 [  4  28 321]]
(of all)
up neutral down
[[1433   25    3]
 [  27  883   36]
 [   4   29 1731]]

<Feature importance>
WTI :  0.1624568267001085
USDKRW :  0.1785443066095856
VIX :  0.14309412474216715
M2 :  0.11798266817233244
1 :  0.022676028423785407
2 :  0.08873983865321977
3 :  0.0589701370687725
4 :  0.09624504179297828
5 :  0.07352380534692021
6 :  0.022713068268770054
7 :  0.010598852935097016
8 :  0.0244553012862632

< AI model: save >


In [2]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit, cross_val_score  # 여기에 cross_val_score를 추가합니다.
from sklearn.metrics import confusion_matrix, accuracy_score
import joblib


# 올바른 파일 경로로 데이터를 불러옵니다.
model_data = pd.read_csv('datasets.csv', encoding='cp949')

# 'Date' 칼럼을 인덱스로 설정하고 datetime 타입으로 변환합니다.
model_data['Date'] = pd.to_datetime(model_data['Date'])
model_data.set_index('Date', inplace=True)

# 데이터를 날짜 기준으로 정렬합니다.
model_data.sort_index(inplace=True)

# 훈련 데이터와 테스트 데이터를 나눌 기준 날짜를 설정합니다.
split_date = model_data.index[int(0.8 * len(model_data))]

# 훈련 데이터와 테스트 데이터로 나눕니다.
train_df = model_data[model_data.index <= split_date]
test_df = model_data[model_data.index > split_date]

# 특성과 레이블을 준비합니다.
X_train = train_df.drop(columns=['forward_stage'])
y_train = train_df['forward_stage']
X_test = test_df.drop(columns=['forward_stage'])
y_test = test_df['forward_stage']

# 모델을 초기화합니다.
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42)

# 하이퍼파라미터 그리드를 정의합니다.
param_dist_rf = {
    'n_estimators': [50, 100, 500],
    'max_leaf_nodes': [20, 30, 40, 50],
    'max_features': [1, 2, 3],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 하이퍼파라미터 튜닝을 위해 RandomizedSearchCV를 사용합니다.
# 시계열 데이터의 특성을 고려하여 TimeSeriesSplit을 사용합니다.
time_split_cv = TimeSeriesSplit(n_splits=10)
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=time_split_cv, random_state=42)
rnd_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터로 모델을 훈련합니다.
best_clf = rnd_search.best_estimator_
best_clf.fit(X_train, y_train)

# 모델을 평가합니다.
train_score = best_clf.score(X_train, y_train)
test_score = best_clf.score(X_test, y_test)

# 10-fold cross-validation의 정확도를 계산합니다.
cv_scores = cross_val_score(best_clf, X_train, y_train, cv=time_split_cv, scoring='accuracy')
cv_mean_score = cv_scores.mean()

# 혼동 행렬을 확인합니다.
y_test_pred = best_clf.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"])
y_all_pred = best_clf.predict(model_data.drop(columns=['forward_stage']))
cm_all = confusion_matrix(model_data['forward_stage'], y_all_pred, labels=["up", "neutral", "down"])

# 특성 중요도를 확인합니다.
feature_importance = list(zip(X_train.columns, best_clf.feature_importances_))

# 모델을 저장합니다.
joblib.dump(best_clf, "separation.pkl")

# 결과를 출력합니다.
print(f"{rnd_search.best_params_}\n")
print("<10-fold cross-validation>")
print("accuracy score mean: ", cv_mean_score)
print("\n<AI model: machine learning done >")
print("accuracy_score of train data(0.8 of sample): ", train_score)
print("accuracy_score of test data(0.2 of sample): ", test_score)
print("\n<Confusion matrix>")
print("(of test)")
print("up", "neutral", "down")
print(cm_test)
print("(of all)")
print("up", "neutral", "down")
print(cm_all)
print("\n<Feature importance>")
for name, score in feature_importance:
    print(name, ": ", score)
print("\n< AI model: save >")


{'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_leaf_nodes': 50, 'max_features': 2, 'max_depth': 15}

<10-fold cross-validation>
accuracy score mean:  0.41650165016501645

<AI model: machine learning done >
accuracy_score of train data(0.8 of sample):  0.8405753670961942
accuracy_score of test data(0.2 of sample):  0.6199040767386091

<Confusion matrix>
(of test)
up neutral down
[[185   0 113]
 [ 24   0 130]
 [ 50   0 332]]
(of all)
up neutral down
[[1235   86  140]
 [ 181  476  289]
 [  93   60 1611]]

<Feature importance>
WTI :  0.1194470854581351
USDKRW :  0.10706590878728613
VIX :  0.09635203262283475
M2 :  0.15077570486899117
1 :  0.03617813015239518
2 :  0.11676609165759229
3 :  0.09410136859910449
4 :  0.12277545968857136
5 :  0.07270371650626584
6 :  0.03736377083656502
7 :  0.014782089943835838
8 :  0.03168864087842287

< AI model: save >


In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.utils import class_weight
import numpy as np

# 데이터 로딩
model_data = pd.read_csv('feature.csv', encoding='cp949')

# 'Date' 칼럼을 인덱스로 설정하고 datetime 타입으로 변환
model_data['Date'] = pd.to_datetime(model_data['Date'])
model_data.set_index('Date', inplace=True)

# 데이터를 날짜 기준으로 정렬
model_data.sort_index(inplace=True)

# 훈련 데이터와 테스트 데이터를 나눌 기준 날짜를 설정
split_date = model_data.index[int(0.8 * len(model_data))]

# 훈련 데이터와 테스트 데이터로 나눔
train_df = model_data[model_data.index <= split_date]
test_df = model_data[model_data.index > split_date]

# 특성과 레이블을 준비
X_train = train_df.drop(columns=['forward_stage'])
y_train = train_df['forward_stage']
X_test = test_df.drop(columns=['forward_stage'])
y_test = test_df['forward_stage']

# 클래스 불균형 문제를 해결하기 위해 클래스 가중치를 계산
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# 모델 초기화
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42, class_weight=class_weight_dict)

# 하이퍼파라미터 그리드를 확장
param_dist_rf = {
    'n_estimators': [50, 100, 200, 500],
    'max_leaf_nodes': [20, 30, 40, 50, 60],
    'max_features': [1, 2, 3, 4],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [class_weight_dict, None]
}

# 하이퍼파라미터 튜닝을 위해 RandomizedSearchCV를 사용
time_split_cv = TimeSeriesSplit(n_splits=10)
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=time_split_cv, random_state=42)
rnd_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터로 모델을 훈련
best_clf = rnd_search.best_estimator_
best_clf.fit(X_train, y_train)

# 모델 평가
train_score = best_clf.score(X_train, y_train)
test_score = best_clf.score(X_test, y_test)

# 10-fold cross-validation의 정확도를 계산
cv_scores = cross_val_score(best_clf, X_train, y_train, cv=time_split_cv, scoring='accuracy')
cv_mean_score = cv_scores.mean()

# 결과 출력 및 모델 저장

# 혼동 행렬을 확인합니다.
y_test_pred = best_clf.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"])
y_all_pred = best_clf.predict(model_data.drop(columns=['forward_stage']))
cm_all = confusion_matrix(model_data['forward_stage'], y_all_pred, labels=["up", "neutral", "down"])

# 특성 중요도를 확인합니다.
feature_importance = list(zip(X_train.columns, best_clf.feature_importances_))

# 모델을 저장합니다.
joblib.dump(best_clf, "separation11.pkl")

# 결과를 출력합니다.
print(f"{rnd_search.best_params_}\n")
print("<10-fold cross-validation>")
print("accuracy score mean: ", cv_mean_score)
print("\n<AI model: machine learning done >")
print("accuracy_score of train data(0.8 of sample): ", train_score)
print("accuracy_score of test data(0.2 of sample): ", test_score)
print("\n<Confusion matrix>")
print("(of test)")
print("up", "neutral", "down")
print(cm_test)
print("(of all)")
print("up", "neutral", "down")
print(cm_all)
print("\n<Feature importance>")
for name, score in feature_importance:
    print(name, ": ", score)
print("\n< AI model: save >")


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py", line 57, in _wrapfunc
    return bound(*args, **kwds)
TypeError: '<' not supported between instances of 'str' and 'int'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  Fil

{'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_leaf_nodes': 60, 'max_features': 1, 'max_depth': 20, 'class_weight': None}

<10-fold cross-validation>
accuracy score mean:  0.4092409240924092

<AI model: machine learning done >
accuracy_score of train data(0.8 of sample):  0.8471681150734193
accuracy_score of test data(0.2 of sample):  0.6546762589928058

<Confusion matrix>
(of test)
up neutral down
[[176   0 122]
 [ 20   0 134]
 [ 12   0 370]]
(of all)
up neutral down
[[1230   86  145]
 [ 175  488  283]
 [  54   55 1655]]

<Feature importance>
WTI :  0.1116420697840777
USDKRW :  0.11494926996927929
VIX :  0.10166662084407262
M2 :  0.1290566254636067
1 :  0.045615676228569095
2 :  0.11209548512539244
3 :  0.09119740362460131
4 :  0.10531184321566138
5 :  0.08868556829703704
6 :  0.04396373247667015
7 :  0.019748764773851294
8 :  0.03606694019718076

< AI model: save >


In [3]:
# -*- coding: utf-8 -*- 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn.metrics as mt 
from sklearn.tree import export_graphviz 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV 
import joblib 
from sklearn.metrics import confusion_matrix 

# Loading the data again after ensuring the correct file path is used
model_data = pd.read_csv('feature.csv', encoding='cp949')

# Setting 'Date' column as index and converting it to datetime
model_data['Date'] = pd.to_datetime(model_data['Date'])
model_data.set_index('Date', inplace=True)

# Sorting data by date
model_data.sort_index(inplace=True)

# Define a split date. As we don't have a user-provided split date, using a placeholder.
# The user should replace it with the actual split date.
split_date = '2022-01-03'  

# Splitting the data into train and test
train_df = model_data[model_data.index <= split_date]
test_df = model_data[model_data.index > split_date]

# Preparing features and labels
X_train = train_df.drop(columns=['forward_stage'])
y_train = train_df['forward_stage']
X_test = test_df.drop(columns=['forward_stage'])
y_test = test_df['forward_stage']

# Initializing the model
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42)

# Define hyperparameter grid
param_dist_rf = {
    'n_estimators': [50, 100, 500],
    'max_leaf_nodes': [20, 30, 40, 50],
    'max_features': [1, 2, 3],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV for hyperparameter tuning
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42)
rnd_search.fit(X_train, y_train)

# Train the model with best parameters
best_clf = rnd_search.best_estimator_
best_clf.fit(X_train, y_train)

# Evaluate the model
train_score = best_clf.score(X_train, y_train)
test_score = best_clf.score(X_test, y_test)

# Confusion matrix
y_test_pred = best_clf.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"])
y_all_pred = best_clf.predict(model_data.drop(columns=['forward_stage']))
cm_all = confusion_matrix(model_data['forward_stage'], y_all_pred, labels=["up", "neutral", "down"])

# Feature importance
feature_importance = list(zip(X_train.columns, best_clf.feature_importances_))

# Save the model
joblib.dump(best_clf, "forecast_model.pkl")

train_score, test_score, cm_test, cm_all, feature_importance


(0.7235901509134234,
 0.5558375634517766,
 array([[  0,   0,  85],
        [  0,   0,  90],
        [  0,   0, 219]], dtype=int64),
 array([[1168,   12,  281],
        [ 260,  131,  555],
        [  87,   24, 1653]], dtype=int64),
 [('WTI', 0.10854812866140856),
  ('USDKRW', 0.1426964236443898),
  ('VIX', 0.16929253948500653),
  ('M2', 0.11800574910278497),
  ('1', 0.0263733766352262),
  ('2', 0.11231968616666016),
  ('3', 0.08294082561505137),
  ('4', 0.1140233703912902),
  ('5', 0.07254457752539362),
  ('6', 0.011181101686514833),
  ('7', 0.008959586691127954),
  ('8', 0.0331146343951458)])

In [4]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
import joblib

# 올바른 파일 경로로 데이터를 불러옵니다.
model_data = pd.read_csv('feature.csv', encoding='cp949')

# 'Date' 칼럼을 인덱스로 설정하고 datetime 타입으로 변환합니다.
model_data['Date'] = pd.to_datetime(model_data['Date'])
model_data.set_index('Date', inplace=True)

# 데이터를 날짜 기준으로 정렬합니다.
model_data.sort_index(inplace=True)

# 훈련 데이터와 테스트 데이터를 나눌 기준 날짜를 설정합니다.
# 사용자가 지정한 split_date가 없기 때문에 데이터셋의 80%에 해당하는 날짜를 자동으로 선택합니다.
split_date = model_data.index[int(0.8 * len(model_data))]

# 훈련 데이터와 테스트 데이터로 나눕니다.
train_df = model_data[model_data.index <= split_date]
test_df = model_data[model_data.index > split_date]

# 특성과 레이블을 준비합니다.
X_train = train_df.drop(columns=['forward_stage'])
y_train = train_df['forward_stage']
X_test = test_df.drop(columns=['forward_stage'])
y_test = test_df['forward_stage']

# 모델을 초기화합니다.
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42)

# 하이퍼파라미터 그리드를 정의합니다.
param_dist_rf = {
    'n_estimators': [50, 100, 500],
    'max_leaf_nodes': [20, 30, 40, 50],
    'max_features': [1, 2, 3],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 하이퍼파라미터 튜닝을 위해 RandomizedSearchCV를 사용합니다.
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42)
rnd_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터로 모델을 훈련합니다.
best_clf = rnd_search.best_estimator_
best_clf.fit(X_train, y_train)

# 모델을 평가합니다.
train_score = best_clf.score(X_train, y_train)
test_score = best_clf.score(X_test, y_test)

# 혼동 행렬을 확인합니다.
y_test_pred = best_clf.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"])
y_all_pred = best_clf.predict(model_data.drop(columns=['forward_stage']))
cm_all = confusion_matrix(model_data['forward_stage'], y_all_pred, labels=["up", "neutral", "down"])

# 특성 중요도를 확인합니다.
feature_importance = list(zip(X_train.columns, best_clf.feature_importances_))

# 모델을 저장합니다.
joblib.dump(best_clf, "separation.pkl")

# 결과를 출력합니다.
train_score, test_score, cm_test, cm_all, feature_importance


(0.704225352112676,
 0.6342925659472423,
 array([[163,   0, 135],
        [ 21,   0, 133],
        [ 16,   0, 366]], dtype=int64),
 array([[1053,   36,  372],
        [ 229,  198,  519],
        [ 113,   23, 1628]], dtype=int64),
 [('WTI', 0.10769227613713275),
  ('USDKRW', 0.1054543026001435),
  ('VIX', 0.10868732316066305),
  ('M2', 0.09405144083259712),
  ('1', 0.04860205681700061),
  ('2', 0.12072185830751007),
  ('3', 0.1091050480849538),
  ('4', 0.12483697609451415),
  ('5', 0.06459268812687957),
  ('6', 0.04384885691719701),
  ('7', 0.02692945277683438),
  ('8', 0.04547772014457407)])

In [ ]:
# Loading the data again with the correct file path this time
model_data = pd.read_csv('feature.csv', encoding='cp949')

# Setting 'Date' column as index and converting it to datetime
model_data['Date'] = pd.to_datetime(model_data['Date'])
model_data.set_index('Date', inplace=True)

# Sorting data by date
model_data.sort_index(inplace=True)

# Define a split date. As we don't have a user-provided split date, using a placeholder.
# The user should replace it with the actual split date.
split_date = '2022-01-03'  

# Splitting the data into train and test based on the split date
train_df = model_data[model_data.index <= split_date]
test_df = model_data[model_data.index > split_date]

# Preparing features and labels
X_train = train_df.drop(columns=['forward_stage'])
y_train = train_df['forward_stage']
X_test = test_df.drop(columns=['forward_stage'])
y_test = test_df['forward_stage']

# Initializing the model
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=1, random_state=42)

# Define hyperparameter grid
param_dist_rf = {
    'n_estimators': [50, 100, 500],
    'max_leaf_nodes': [20, 30, 40, 50],
    'max_features': [1, 2, 3],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# RandomizedSearchCV for hyperparameter tuning
rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42)
rnd_search.fit(X_train, y_train)

# Train the model with best parameters
best_clf = rnd_search.best_estimator_
best_clf.fit(X_train, y_train)

# Evaluate the model
train_score = best_clf.score(X_train, y_train)
test_score = best_clf.score(X_test, y_test)

# Confusion matrix
y_test_pred = best_clf.predict(X_test)
cm_test = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"])
y_all_pred = best_clf.predict(model_data.drop(columns=['forward_stage']))
cm_all = confusion_matrix(model_data['forward_stage'], y_all_pred, labels=["up", "neutral", "down"])

# Feature importance
feature_importance = list(zip(X_train.columns, best_clf.feature_importances_))

# Save the model
joblib.dump(best_clf, "1.pkl")

train_score, test_score, cm_test, cm_all, feature_importance


In [3]:
# # -*- coding: utf-8 -*-
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import sklearn.metrics as mt
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import StratifiedShuffleSplit, RandomizedSearchCV, cross_val_score
# import joblib
# from sklearn.metrics import confusion_matrix

# # 1. 수정된 CSV 데이터 로드
# model_data = pd.read_csv("3.csv")

# # 'Date' 컬럼을 인덱스로 설정
# model_data.set_index('Date', inplace=True)

# # 2. 피처와 라벨의 완전한 데이터 생성
# # 'forward_stage' 컬럼을 제외한 나머지를 피처로 사용
# X = model_data.drop(columns=['forward_stage'])
# # 'forward_stage' 컬럼을 라벨로 사용
# y = model_data['forward_stage']

# # 과거 데이터만 선택
# X_past = X[y.notna()]
# y_past = y[y.notna()]

# # 3. 데이터를 훈련 세트와 테스트 세트로 분할
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# for train_index, test_index in sss.split(X_past, y_past):
#     X_train, X_test = X_past.iloc[train_index,], X_past.iloc[test_index,]
#     y_train, y_test = y_past.iloc[train_index], y_past.iloc[test_index]

# # 4. 모델 파인튜닝: 최적의 하이퍼파라미터 찾기
# rnd_clf = RandomForestClassifier(n_jobs=1, random_state=42)  # n_jobs를 1로 설정하여 멀티프로세싱을 사용하지 않음

# # 탐색할 하이퍼파라미터 설정
# param_dist_rf = {
#     'n_estimators': [50, 100, 500],  # 트리의 개수
#     'max_leaf_nodes': [20, 30, 40, 50],  # 리프 노드의 최대 개수
#     'max_features': [1, 2, 3],  # 최대 피처 개수
#     'max_depth': [5, 10, 15],  # 최대 깊이
#     'min_samples_split': [2, 5, 10],  # 노드를 분할하기 위한 최소 샘플 개수
#     'min_samples_leaf': [1, 2, 4]  # 리프 노드에 있어야 하는 최소 샘플 개수
# }

# # RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
# rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42)
# rnd_search.fit(X_train, y_train)
# print("최적의 파라미터: ", rnd_search.best_params_)

# # 5. 모델 훈련 및 K-겹 교차 검증을 사용한 평가
# rnd_scores = cross_val_score(rnd_clf, X_train, y_train, scoring="accuracy", cv=10)
# print("\n<10-겹 교차 검증>")
# print("정확도 평균: ", rnd_scores.mean())

# # 6. 최종 모델 훈련
# rnd_clf.fit(X_train, y_train)
# print("\n<AI 모델: 학습 완료>")
# print("훈련 데이터 정확도(샘플의 0.8): ", rnd_clf.score(X_train, y_train))

# # 7. 테스트 데이터에서 모델 평가
# print("테스트 데이터 정확도(샘플의 0.2): ", rnd_clf.score(X_test, y_test))

# # 8. 혼동 행렬 확인
# y_test_pred = rnd_clf.predict(X_test)
# cm1 = confusion_matrix(y_test, y_test_pred, labels=["up", "neutral", "down"])
# print("\n<혼동 행렬>")
# print("(테스트 데이터)")
# print("상승", "보합", "하락")
# print(cm1)
# cm2 = confusion_matrix(y_past, rnd_clf.predict(X_past), labels=["up", "neutral", "down"])
# print("(전체 데이터)")
# print("상승", "보합", "하락")
# print(cm2)

# # 9. 피처 중요도 확인
# print("\n<피처 중요도>")
# for name, score in zip(X.columns, rnd_clf.feature_importances_):
#     print(name, ": ", score)

# # 10. 백테스팅을 위한 예측 데이터 생성
# y_prediction = rnd_clf.predict(X)
# y_pred = pd.Series(y_prediction, index=y.index)

# # 11. 모델 저장
# joblib.dump(rnd_clf, "forecast_model.pkl")
# print("\n< AI 모델: 저장 완료>")


In [5]:
y_pred

Date
07-01-02    neutral
07-01-03    neutral
07-01-04         up
07-01-05         up
07-01-08         up
             ...   
16-11-24         up
16-12-26         up
17-01-02         up
17-01-16         up
17-02-20         up
Length: 4303, dtype: object

In [ ]:
# ml2.py code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier

# 1. 모델 로드
rnd_clf = joblib.load("forecast_model.pkl")
print("\n< AI model: load >")

# 2. new daily raw data 가져오기
model_data = pd.read_csv("3.csv")

# 'forward_stage' 컬럼을 숫자로 변환합니다.
categories = model_data['forward_stage'].astype('category')
model_data['forward_stage'] = categories.cat.codes

# X는 'forward_stage'와 'Date'를 제외한 모든 컬럼을 포함해야 합니다.
X = model_data.drop(columns=['forward_stage', 'Date'])
X.columns = X.columns.astype(str)  # Set feature names to avoid warning
y = model_data["forward_stage"]

# y가 NaN이 아닌 행만 선택합니다.
X_past = X[y.notna()]
y_past = y[y.notna()]

# 3. new daily raw data 전체 학습
rnd_clf.fit(X_past, y_past)
print("\n< AI model: machine learning done >")
print("accuracy_score of whole data: ", rnd_clf.score(X_past, y_past))

# 4. 현재(마지막) 데이터 표시
print("\n<Current status>")
for col, score in zip(X.columns, X.iloc[-1]):
    print("{:20} : {:>8.3f}".format(col, score))

X_current = np.array(X.iloc[-1]).reshape(1, -1)

# 5. 현재 전망
print("\n< AI model: forecasting >")
y_current_pred = rnd_clf.predict(X_current)
print("forecast: ", categories.cat.categories[y_current_pred[0]])

# 현재전망의 확률표
prob_current = rnd_clf.predict_proba(X_current)
y_names = rnd_clf.classes_
print("\n[class] : [prob]")
for name, prob in zip(categories.cat.categories[y_names], prob_current[0]):
    print("{:7} : {:.2f}".format(name, prob))

# # 6. 2023년 일별 전망치의 확률 변화
# # 전기간 전망치 확률 데이터생성
# prob = rnd_clf.predict_proba(X)
# prob_df = pd.DataFrame(prob, columns=categories.cat.categories)

# # '2023'이라는 컬럼이나 인덱스는 없으므로, 'Date' 컬럼에서 '2023'년에 해당하는 데이터를 선택해야 합니다.
# model_data['Year'] = pd.to_datetime(model_data['Date'], format='%y-%m-%d').dt.year
# prob_2023 = prob_df[model_data['Year'] == 2023]

# # '2023'년에 해당하는 날짜를 가져옵니다.
# dates_2023 = model_data.loc[model_data['Year'] == 2023, 'Date']

# plt.bar(dates_2023, prob_2023['up'], label='up', color='r')
# plt.bar(dates_2023, prob_2023['neutral'], label='neutral', color='g', bottom=prob_2023['up'])
# plt.bar(dates_2023, prob_2023['down'], label='down', color='b', bottom=prob_2023[['up', 'neutral']].sum(axis=1))
# plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
# plt.legend()
# plt.show()


In [21]:
# -*- coding: utf-8 -*- 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
# -1. MDD 함수 정의
def MDD(list_values): 
 mdd_value = 0 
 for i in range(1, len(list_values)): 
    bw_max = max(list_values[:i]) 
    curr = list_values[i] 
    mdd = curr / bw_max - 1 
 if mdd < mdd_value: 
    mdd_value = mdd 
 return mdd_value 
# 0. 사후방향성 클래스를 수치로 전환하는 함수 정의 (up=1, neutral=0, down=-1) 
def convert_num(pred): 
    pred_num = np.empty(len(pred)) 
    pred_num[pred=='up']=1 
    pred_num[pred=='neutral']=0 
    pred_num[pred=='down']=-1 
    pred_num[pred.isna()]=np.NaN 
    
    return pred_num 


In [22]:
# backtest.py code
# 1. 월간 시장수익률 데이터 가져오기
# 데이터는 일자/월수익률(원수치)/코스피지수/월말하루전영업일로 구성
kdata = pd.read_csv("investing.csv")
# 2. AI모델로 예측한 예측정보를 가져오기
# 기존 AI모델 학습 프로세스에서 만들어낸 y_pred 데이터를 조회함. 
# 매월말에 예측했던 투자의견을 가져옴. 단, 실제 투자를 위해서, 월말하루전영업일 기준 자료를 가져옴
# kdata['stage']='' 
kdata['stage'] = y_pred #말일자 아님
for index in kdata.index: 
 kdata.loc[index,'stage']=y_pred[kdata.loc[index,'before_last']] 
# 3. 전월말 투자의견 열 생성
kdata['pre_stage']= kdata['stage'].shift(1) 
kdata['port_return']=0 


KeyError: '2007-04-29'

In [ ]:

# 4. 전략 수익률 생성
# 전월말 투자의견이 상승이면 코스피 long, 보합이면 Cash, 하락이면 코스피 Short 실행. 
# 해당 전략에 따라 포트 월별수익률(port_return) 생성
kdata.loc[kdata['pre_stage']=='up', 'port_return'] = kdata['m_return']*1 
kdata.loc[kdata['pre_stage'].isna(), 'port_return'] = kdata['m_return']*1 
kdata.loc[kdata['pre_stage']=='neutral', 'port_return'] = 0 
kdata.loc[kdata['pre_stage']=='down', 'port_return'] = kdata['m_return']*-1 
# 코스피와 모델포트폴리오의 누적수익률(1에서 시작하는 인덱스 형태) 생성
kdata['kospi_cumul']=(1+kdata['m_return']).cumprod() 
kdata['port_cumul']=(1+kdata['port_return']).cumprod() 
# 5. 백테스팅 결과 기록(CAGR, 변동성, Sharpe ratio, MDD) 
my_back = {'months':len(kdata)} 

my_back['k_cumul_return_idx']=kdata['kospi_cumul'][-1] 
my_back['k_cumul_return_pct']=(my_back['k_cumul_return_idx']-1)*100 
my_back['k_cagr']=(my_back['k_cumul_return_idx']**(12/my_back['months']))-1 
my_back['k_cagr_pct']=my_back['k_cagr']*100 
my_back['k_vol_pct']=np.std(kdata['m_return'])*np.sqrt(12)*100 
my_back['k_Sharpe']=my_back['k_cagr_pct']/my_back['k_vol_pct'] 
my_back['k_MDD']=MDD(kdata['kospi_cumul'])*100 

my_back['port_cumul_return_idx']=kdata['port_cumul'][-1] 
my_back['port_cumul_return_pct']=(my_back['port_cumul_return_idx']-1)*100 
my_back['port_cagr']=(my_back['port_cumul_return_idx']**(12/my_back['months']))-1 
my_back['port_cagr_pct']=my_back['port_cagr']*100 
my_back['port_vol_pct']=np.std(kdata['port_return'])*np.sqrt(12)*100 
my_back['port__Sharpe']=my_back['port_cagr_pct']/my_back['port_vol_pct'] 
my_back['port_MDD']=MDD(kdata['port_cumul'])*100 

# 6. 백테스팅 결과 출력하기
print("<Backtesting result>") 
for key, value in my_back.items(): 
 print("{:22}: {:>8.3f}".format(key, value)) 
 
# 포트폴리오 누적수익률 그래프
kdata['port_cumul'].plot() 
plt.title('Portfolio performance index') 
plt.ylabel('\'02/12/31 = 1') 
plt.show() 

# 7. 월말 모델전망치와 실제결과치 출력
# 월말 실제결과치 입력
for index in kdata.index: 

 kdata.loc[index,'real_stage']=y[kdata.loc[index,'before_last']] 
# 사후방향성 클래스를 수치로 변환
kdata['stage_num']=convert_num(kdata['stage']) 
kdata['real_stage_num']=convert_num(kdata['real_stage']) 

# 전망치와 결과치의 그래프 출력
kdata.plot(y=['stage_num', 'real_stage_num'], label=['model forecast','real direction']) 
plt.title('Model forecast vs. Real direction') 
plt.ylabel('up=1, neutral=0, down=-1') 
plt.show() 